# **Análise Exploratória de Salários em Carreiras de Dados**

## **Contextualização**

Nos últimos anos, a área de Ciência de Dados cresceu de forma acelerada, se tornando uma das mais promissoras no mercado de tecnologia. Profissionais como Data Analysts e Data Scientists passaram a ser cada vez mais valorizados, mas ainda existe muita dúvida sobre como os salários variam entre cargos, níveis de experiência e localização.

Este projeto tem como objetivo realizar uma Análise Exploratória de Dados (EDA) sobre salários de profissionais da área de dados, utilizando um dataset público do Kaggle. A análise busca responder perguntas como:

1- Qual a distribuição de salários em dólar para profissionais de dados?

2- Como variam os salários entre Data Analysts e Data Scientists?

3- Qual o impacto do nível de experiência (junior, pleno, sênior, executivo) na remuneração?

4- Há diferenças significativas dependendo da localização da empresa?

## **Objetivo**

Além de explorar o dataset, este projeto também tem como propósito:

1- Exercitar técnicas de manipulação de dados com Pandas.

2- Criar visualizações claras com Matplotlib e Seaborn.

3- Extrair insights relevantes que podem ser úteis para quem deseja ingressar ou crescer na área de dados.